## Step 1: Load the Data

In [0]:
# (EDIT the four lines below if needed)
storage_account = "covidprojectdata"
tenant_id       = "4e8362c7-dae8-414e-91a1-198b637b91e5"
client_id       = "d274dbd4-c929-45ce-b9e3-762cee2e5642"
client_secret   = dbutils.secrets.get(scope="new-scope",key="DatabricksSPSecret")

# 1) Set up OAuth for ABFS
spark.conf.set(
    f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net",
    "OAuth"
)
spark.conf.set(
    f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net",
    "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider"
)
spark.conf.set(
    f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net",
    client_id
)
spark.conf.set(
    f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net",
    client_secret
)
spark.conf.set(
    f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net",
    f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
)




In [0]:
display(dbutils.fs.ls(f"abfss://covid-project-data@{storage_account}.dfs.core.windows.net/"))


path,name,size,modificationTime
abfss://covid-project-data@covidprojectdata.dfs.core.windows.net/raw-data/,raw-data/,0,1748571220000
abfss://covid-project-data@covidprojectdata.dfs.core.windows.net/transformed-data/,transformed-data/,0,1748571246000


In [0]:
death = spark.read.csv(
  f"abfss://covid-project-data@covidprojectdata.dfs.core.windows.net/raw-data/casesdeaths.csv",
  header=True,
  inferSchema=True
)
display(death.limit(10))

pruid,prname,prnameFR,date,reporting_week,reporting_year,update,totalcases,numtotal_last7,ratecases_total,numdeaths,numdeaths_last7,ratedeaths,ratecases_last7,ratedeaths_last7,numtotal_last14,numdeaths_last14,ratetotal_last14,ratedeaths_last14,avgcases_last7,avgincidence_last7,avgdeaths_last7,avgratedeaths_last7
59,British Columbia,Colombie-Britannique,2020-02-08,6,2020,1.0,4,3,0.07,0,0.0,0.0,0.05,0.0,4,0.0,0.07,0.0,0.43,0.01,0.0,0.0
48,Alberta,Alberta,2020-02-08,6,2020,1.0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
47,Saskatchewan,Saskatchewan,2020-02-08,6,2020,1.0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
46,Manitoba,Manitoba,2020-02-08,6,2020,1.0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
35,Ontario,Ontario,2020-02-08,6,2020,1.0,4,1,0.03,0,0.0,0.0,0.01,0.0,1,0.0,0.01,0.0,0.14,0,0.0,0.0
24,Quebec,Québec,2020-02-08,6,2020,1.0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
10,Newfoundland and Labrador,Terre-Neuve-et-Labrador,2020-02-08,6,2020,1.0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
13,New Brunswick,Nouveau-Brunswick,2020-02-08,6,2020,1.0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
12,Nova Scotia,Nouvelle-Écosse,2020-02-08,6,2020,1.0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
11,Prince Edward Island,Île-du-Prince-Édouard,2020-02-08,6,2020,1.0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0


In [0]:
vaccination = spark.read.csv(
  f"abfss://covid-project-data@covidprojectdata.dfs.core.windows.net/raw-data/vaccination.csv",
  header=True,
  inferSchema=True
)
display(vaccination.limit(10))

pruid,prename,report_date,numtotal_all_administered,as_of_date,numtotal_dose1_administered,numtotal_dose2_administered,numtotal_dose3_administered,numtotal_dose4_administered,updated
24,Quebec,2021-01-19,164053,2021-01-18,null,null,null,null,1
35,Ontario,2021-01-19,262992,2021-01-18,246799,16193,0,0,1
59,British Columbia,2021-01-19,92369,2021-01-19,null,null,null,null,1
48,Alberta,2021-01-19,92315,2021-01-18,null,null,null,null,1
47,Saskatchewan,2021-01-19,24575,2021-01-18,null,null,null,null,1
12,Nova Scotia,2021-01-19,8520,2021-01-19,null,null,null,null,1
46,Manitoba,2021-01-19,17751,2021-01-18,null,null,null,null,1
13,New Brunswick,2021-01-19,6241,2021-01-09,6241,0,0,null,1
10,Newfoundland and Labrador,2021-01-19,5291,2021-01-13,null,null,null,null,1
11,Prince Edward Island,2021-01-19,5680,2021-01-16,4293,1387,null,null,1


In [0]:
provpop = spark.read.csv(
  f"abfss://covid-project-data@covidprojectdata.dfs.core.windows.net/raw-data/prov_pop.csv",
  header=True,
  inferSchema=True
)
display(provpop.limit(10))

REF_DATE,GEO,DGUID,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
2018-01-01,Canada,2016A000011124,Persons,249,units,0,v1,1,36801579,null,null,null,0
2018-04-01,Canada,2016A000011124,Persons,249,units,0,v1,1,36903671,null,null,null,0
2018-07-01,Canada,2016A000011124,Persons,249,units,0,v1,1,37072620,null,null,null,0
2018-10-01,Canada,2016A000011124,Persons,249,units,0,v1,1,37259485,null,null,null,0
2019-01-01,Canada,2016A000011124,Persons,249,units,0,v1,1,37336956,null,null,null,0
2019-04-01,Canada,2016A000011124,Persons,249,units,0,v1,1,37437243,null,null,null,0
2019-07-01,Canada,2016A000011124,Persons,249,units,0,v1,1,37618495,null,null,null,0
2019-10-01,Canada,2016A000011124,Persons,249,units,0,v1,1,37828162,null,null,null,0
2020-01-01,Canada,2016A000011124,Persons,249,units,0,v1,1,37928208,null,null,null,0
2020-04-01,Canada,2016A000011124,Persons,249,units,0,v1,1,38006941,null,null,null,0


## STEP 2: Clean the data

####Steps to Follow

1. **Inspect Raw Data:**
   - Review schema, sample rows, and counts of null/blank/“-” values.

2. **Drop Unnecessary Columns:**
   - Remove columns not needed (e.g., `prnameFR`).

3. **Clean Data:**
   - Replace "-" and blank strings with null.
   - Cast to correct numeric types.

4. **Handle Missing Data:**
   - Drop rows missing critical keys (`pruid`, `reporting_year`, `reporting_week`, `date`).
   - Drop rows with null in required metrics or decide to fill with zero if appropriate.

5. **Filter Data:**
   - Remove rows with negative values in metrics.

6. **Remove Duplicates:**
   - Eliminate duplicate rows based on the natural key (`pruid`, `reporting_year`, `reporting_week`).

### 2.1 Inspect Raw Data

- Check schema and sample rows.
- Count how many null / blank / "-" values appear in key columns.


In [0]:
from pyspark.sql.functions import col, when, trim, sum as _sum

# 2.1.1: Show schema & first 5 rows
death.printSchema()
death.show(5, truncate=False)

# 2.1.2: Count null/blank/"-" in critical columns
cols_to_check = [
  "pruid", "prname", "prnameFR",
  "date", "reporting_week", "reporting_year",
  "numtotal_last7", "numdeaths_last7",
  "ratecases_last7", "ratedeaths_last7"
]

exprs = []
for c in cols_to_check:
    exprs.append(
        _sum(
            when(
                col(c).isNull() | (trim(col(c)) == "") | (col(c) == "-"),
                1
            ).otherwise(0)
        ).alias(f"{c}_badCount")
    )

death.select(exprs).show(truncate=False)


root
 |-- pruid: integer (nullable = true)
 |-- prname: string (nullable = true)
 |-- prnameFR: string (nullable = true)
 |-- date: date (nullable = true)
 |-- reporting_week: integer (nullable = true)
 |-- reporting_year: integer (nullable = true)
 |-- update: double (nullable = true)
 |-- totalcases: string (nullable = true)
 |-- numtotal_last7: string (nullable = true)
 |-- ratecases_total: string (nullable = true)
 |-- numdeaths: integer (nullable = true)
 |-- numdeaths_last7: double (nullable = true)
 |-- ratedeaths: double (nullable = true)
 |-- ratecases_last7: string (nullable = true)
 |-- ratedeaths_last7: double (nullable = true)
 |-- numtotal_last14: string (nullable = true)
 |-- numdeaths_last14: double (nullable = true)
 |-- ratetotal_last14: string (nullable = true)
 |-- ratedeaths_last14: double (nullable = true)
 |-- avgcases_last7: string (nullable = true)
 |-- avgincidence_last7: string (nullable = true)
 |-- avgdeaths_last7: double (nullable = true)
 |-- avgratedeath

## STEP 2: Clean the **death** Table (keeping more columns)

We will keep all of these columns:

- `pruid`
- `prname`
- `prnameFR`
- `date`
- `reporting_week`
- `reporting_year`
- `update`
- `totalcases`
- `numtotal_last7`
- `ratecases_total`
- `numdeaths`
- `numdeaths_last7`
- `ratedeaths`
- `ratecases_last7`
- `ratedeaths_last7`
- `numtotal_last14`
- `numdeaths_last14`
- `ratetotal_last14`
- `ratedeaths_last14`
- `avgcases_last7`
- `avgincidence_last7`
- `avgdeaths_last7`
- `avgratedeaths_last7`


In [0]:
from pyspark.sql.functions import col, when, trim, to_date, lit
from pyspark.sql.types import IntegerType, DoubleType

# 2.1: Select all desired columns from raw `death`
cols_keep_death = [
    "pruid",
    "prname",
    "prnameFR",
    "date",
    "reporting_week",
    "reporting_year",
    "update",
    "totalcases",
    "numtotal_last7",
    "ratecases_total",
    "numdeaths",
    "numdeaths_last7",
    "ratedeaths",
    "ratecases_last7",
    "ratedeaths_last7",
    "numtotal_last14",
    "numdeaths_last14",
    "ratetotal_last14",
    "ratedeaths_last14",
    "avgcases_last7",
    "avgincidence_last7",
    "avgdeaths_last7",
    "avgratedeaths_last7"
]

death_trimmed = death.select(*cols_keep_death)

# 2.2: Inspect schema and sample rows
death_trimmed.printSchema()
display(death_trimmed.limit(5))


root
 |-- pruid: integer (nullable = true)
 |-- prname: string (nullable = true)
 |-- prnameFR: string (nullable = true)
 |-- date: date (nullable = true)
 |-- reporting_week: integer (nullable = true)
 |-- reporting_year: integer (nullable = true)
 |-- update: double (nullable = true)
 |-- totalcases: string (nullable = true)
 |-- numtotal_last7: string (nullable = true)
 |-- ratecases_total: string (nullable = true)
 |-- numdeaths: integer (nullable = true)
 |-- numdeaths_last7: double (nullable = true)
 |-- ratedeaths: double (nullable = true)
 |-- ratecases_last7: string (nullable = true)
 |-- ratedeaths_last7: double (nullable = true)
 |-- numtotal_last14: string (nullable = true)
 |-- numdeaths_last14: double (nullable = true)
 |-- ratetotal_last14: string (nullable = true)
 |-- ratedeaths_last14: double (nullable = true)
 |-- avgcases_last7: string (nullable = true)
 |-- avgincidence_last7: string (nullable = true)
 |-- avgdeaths_last7: double (nullable = true)
 |-- avgratedeath

pruid,prname,prnameFR,date,reporting_week,reporting_year,update,totalcases,numtotal_last7,ratecases_total,numdeaths,numdeaths_last7,ratedeaths,ratecases_last7,ratedeaths_last7,numtotal_last14,numdeaths_last14,ratetotal_last14,ratedeaths_last14,avgcases_last7,avgincidence_last7,avgdeaths_last7,avgratedeaths_last7
59,British Columbia,Colombie-Britannique,2020-02-08,6,2020,1.0,4,3,0.07,0,0.0,0.0,0.05,0.0,4,0.0,0.07,0.0,0.43,0.01,0.0,0.0
48,Alberta,Alberta,2020-02-08,6,2020,1.0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
47,Saskatchewan,Saskatchewan,2020-02-08,6,2020,1.0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
46,Manitoba,Manitoba,2020-02-08,6,2020,1.0,0,0,0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
35,Ontario,Ontario,2020-02-08,6,2020,1.0,4,1,0.03,0,0.0,0.0,0.01,0.0,1,0.0,0.01,0.0,0.14,0,0.0,0.0


In [0]:
death_trimmed.count()

3630

### 2.3 Replace `"-"` or blank strings with `null`, then cast to correct types.

- Integer fields:  
  `pruid`, `reporting_week`, `reporting_year`, `update`, `totalcases`,  
  `numtotal_last7`, `numdeaths`, `numdeaths_last7`, `numtotal_last14`, `numdeaths_last14`, `avgcases_last7`, `avgincidence_last7`, `avgdeaths_last7`, `avgratedeaths_last7`  

- Double fields (rates):  
  `ratecases_total`, `ratedeaths`, `ratecases_last7`, `ratedeaths_last7`, `ratetotal_last14`, `ratedeaths_last14`  

- Date field:  
  `date` (format `yyyy-MM-dd`)  

- String fields remain as is:  
  `prname`, `prnameFR`


In [0]:
# 2.3.1: Replace "-" or "" with null for each column that needs casting
tmp = death_trimmed
for c in [
    "pruid",
    "reporting_week",
    "reporting_year",
    "update",
    "totalcases",
    "numtotal_last7",
    "ratecases_total",
    "numdeaths",
    "numdeaths_last7",
    "ratedeaths",
    "ratecases_last7",
    "ratedeaths_last7",
    "numtotal_last14",
    "numdeaths_last14",
    "ratetotal_last14",
    "ratedeaths_last14",
    "avgcases_last7",
    "avgincidence_last7",
    "avgdeaths_last7",
    "avgratedeaths_last7"
]:
    tmp = tmp.withColumn(
        c,
        when((trim(col(c)) == "") | (col(c) == "-"), None).otherwise(col(c))
    )

# 2.3.2: Cast columns to correct types
death_cast = tmp \
    .withColumn("pruid", col("pruid").cast(IntegerType())) \
    .withColumn("reporting_week", col("reporting_week").cast(IntegerType())) \
    .withColumn("reporting_year", col("reporting_year").cast(IntegerType())) \
    .withColumn("update", col("update").cast(IntegerType())) \
    .withColumn("totalcases", col("totalcases").cast(IntegerType())) \
    .withColumn("numtotal_last7", col("numtotal_last7").cast(IntegerType())) \
    .withColumn("ratecases_total", col("ratecases_total").cast(DoubleType())) \
    .withColumn("numdeaths", col("numdeaths").cast(IntegerType())) \
    .withColumn("numdeaths_last7", col("numdeaths_last7").cast(IntegerType())) \
    .withColumn("ratedeaths", col("ratedeaths").cast(DoubleType())) \
    .withColumn("ratecases_last7", col("ratecases_last7").cast(DoubleType())) \
    .withColumn("ratedeaths_last7", col("ratedeaths_last7").cast(DoubleType())) \
    .withColumn("numtotal_last14", col("numtotal_last14").cast(IntegerType())) \
    .withColumn("numdeaths_last14", col("numdeaths_last14").cast(IntegerType())) \
    .withColumn("ratetotal_last14", col("ratetotal_last14").cast(DoubleType())) \
    .withColumn("ratedeaths_last14", col("ratedeaths_last14").cast(DoubleType())) \
    .withColumn("avgcases_last7", col("avgcases_last7").cast(IntegerType())) \
    .withColumn("avgincidence_last7", col("avgincidence_last7").cast(IntegerType())) \
    .withColumn("avgdeaths_last7", col("avgdeaths_last7").cast(IntegerType())) \
    .withColumn("avgratedeaths_last7", col("avgratedeaths_last7").cast(IntegerType())) \
    .withColumn("date", to_date(col("date"), "yyyy-MM-dd"))


In [0]:
death_cast.printSchema()
display(death_cast.limit(5))

root
 |-- pruid: integer (nullable = true)
 |-- prname: string (nullable = true)
 |-- prnameFR: string (nullable = true)
 |-- date: date (nullable = true)
 |-- reporting_week: integer (nullable = true)
 |-- reporting_year: integer (nullable = true)
 |-- update: integer (nullable = true)
 |-- totalcases: integer (nullable = true)
 |-- numtotal_last7: integer (nullable = true)
 |-- ratecases_total: double (nullable = true)
 |-- numdeaths: integer (nullable = true)
 |-- numdeaths_last7: integer (nullable = true)
 |-- ratedeaths: double (nullable = true)
 |-- ratecases_last7: double (nullable = true)
 |-- ratedeaths_last7: double (nullable = true)
 |-- numtotal_last14: integer (nullable = true)
 |-- numdeaths_last14: integer (nullable = true)
 |-- ratetotal_last14: double (nullable = true)
 |-- ratedeaths_last14: double (nullable = true)
 |-- avgcases_last7: integer (nullable = true)
 |-- avgincidence_last7: integer (nullable = true)
 |-- avgdeaths_last7: integer (nullable = true)
 |-- avg

pruid,prname,prnameFR,date,reporting_week,reporting_year,update,totalcases,numtotal_last7,ratecases_total,numdeaths,numdeaths_last7,ratedeaths,ratecases_last7,ratedeaths_last7,numtotal_last14,numdeaths_last14,ratetotal_last14,ratedeaths_last14,avgcases_last7,avgincidence_last7,avgdeaths_last7,avgratedeaths_last7
59,British Columbia,Colombie-Britannique,2020-02-08,6,2020,1,4,3,0.07,0,0,0.0,0.05,0.0,4,0,0.07,0.0,0,0,0,0
48,Alberta,Alberta,2020-02-08,6,2020,1,0,0,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0
47,Saskatchewan,Saskatchewan,2020-02-08,6,2020,1,0,0,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0
46,Manitoba,Manitoba,2020-02-08,6,2020,1,0,0,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0
35,Ontario,Ontario,2020-02-08,6,2020,1,4,1,0.03,0,0,0.0,0.01,0.0,1,0,0.01,0.0,0,0,0,0


### 2.4 Handle Missing Data

- Drop rows missing critical keys: `pruid`, `reporting_year`, `reporting_week`, `date`.
- For numeric metrics, if `null` → fill with 0 (since a missing count usually means zero for that metric).


In [0]:
# 2.4.1: Drop rows missing mandatory keys
mandatory_cols = ["pruid", "reporting_year", "reporting_week", "date"]
death_nonull = death_cast.na.drop(subset=mandatory_cols)

# 2.4.2: Fill nulls with zero for all numeric metric columns
death_filled = death_nonull.fillna({
    "update": 0,
    "totalcases": 0,
    "numtotal_last7": 0,
    "ratecases_total": 0.0,
    "numdeaths": 0,
    "numdeaths_last7": 0,
    "ratedeaths": 0.0,
    "ratecases_last7": 0.0,
    "ratedeaths_last7": 0.0,
    "numtotal_last14": 0,
    "numdeaths_last14": 0,
    "ratetotal_last14": 0.0,
    "ratedeaths_last14": 0.0,
    "avgcases_last7": 0,
    "avgincidence_last7": 0,
    "avgdeaths_last7": 0,
    "avgratedeaths_last7": 0
})


### 2.5 Filter Out Negative Values

Remove any row where any of the numeric count columns is negative, or rate is negative.


In [0]:
death_filtered = death_filled \
    .filter((col("totalcases") >= 0)
            & (col("numtotal_last7") >= 0)
            & (col("numdeaths") >= 0)
            & (col("numdeaths_last7") >= 0)
            & (col("numtotal_last14") >= 0)
            & (col("numdeaths_last14") >= 0)
            & (col("avgcases_last7") >= 0)
            & (col("avgincidence_last7") >= 0)
            & (col("avgdeaths_last7") >= 0)
            & (col("avgratedeaths_last7") >= 0)) \
    .filter((col("ratecases_total") >= 0.0)
            & (col("ratedeaths") >= 0.0)
            & (col("ratecases_last7") >= 0.0)
            & (col("ratedeaths_last7") >= 0.0)
            & (col("ratetotal_last14") >= 0.0)
            & (col("ratedeaths_last14") >= 0.0))


### 2.6 Remove Duplicates Based on Composite Key

Natural key is `(pruid, reporting_year, reporting_week)`. Drop duplicates, keep the first.


In [0]:
death_clean = death_filtered.dropDuplicates(["pruid", "reporting_year", "reporting_week"])


### 2.7 Final Check

Verify schema and row count in `death_clean`.


In [0]:
death_clean.printSchema()
print("Rows in death_clean:", death_clean.count())
display(death_clean.limit(5))


root
 |-- pruid: integer (nullable = true)
 |-- prname: string (nullable = true)
 |-- prnameFR: string (nullable = true)
 |-- date: date (nullable = true)
 |-- reporting_week: integer (nullable = true)
 |-- reporting_year: integer (nullable = true)
 |-- update: integer (nullable = false)
 |-- totalcases: integer (nullable = false)
 |-- numtotal_last7: integer (nullable = false)
 |-- ratecases_total: double (nullable = false)
 |-- numdeaths: integer (nullable = false)
 |-- numdeaths_last7: integer (nullable = false)
 |-- ratedeaths: double (nullable = false)
 |-- ratecases_last7: double (nullable = false)
 |-- ratedeaths_last7: double (nullable = false)
 |-- numtotal_last14: integer (nullable = false)
 |-- numdeaths_last14: integer (nullable = false)
 |-- ratetotal_last14: double (nullable = false)
 |-- ratedeaths_last14: double (nullable = false)
 |-- avgcases_last7: integer (nullable = false)
 |-- avgincidence_last7: integer (nullable = false)
 |-- avgdeaths_last7: integer (nullable =

pruid,prname,prnameFR,date,reporting_week,reporting_year,update,totalcases,numtotal_last7,ratecases_total,numdeaths,numdeaths_last7,ratedeaths,ratecases_last7,ratedeaths_last7,numtotal_last14,numdeaths_last14,ratetotal_last14,ratedeaths_last14,avgcases_last7,avgincidence_last7,avgdeaths_last7,avgratedeaths_last7
12,Nova Scotia,Nouvelle-Écosse,2020-03-07,10,2020,1,0,0,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0
12,Nova Scotia,Nouvelle-Écosse,2020-05-16,20,2020,1,1036,27,97.86,53,8,5.01,2.55,0.76,76,19,7.18,1.79,3,0,1,0
62,Nunavut,Nunavut,2020-05-23,21,2020,1,0,0,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0.0,0,0,0,0
48,Alberta,Alberta,2020-09-19,38,2020,1,16576,994,353.03,253,1,5.39,21.17,0.02,1882,9,40.08,0.19,142,3,0,0
1,Canada,Canada,2021-03-06,9,2021,0,880559,20509,2196.03,21848,226,54.49,51.15,0.56,41280,479,102.95,1.19,2929,7,32,0


### 3.1 Columns to Keep

We will keep:

- `pruid`
- `prename`
- `report_date`
- `as_of_date`
- `numtotal_all_administered`
- `numtotal_dose1_administered`
- `numtotal_dose2_administered`
- `numtotal_dose3_administered`
- `numtotal_dose4_administered`
- `updated`


In [0]:
# 3.1: Select desired columns from raw `vaccination`
cols_keep_vacc = [
    "pruid",
    "prename",
    "report_date",
    "as_of_date",
    "numtotal_all_administered",
    "numtotal_dose1_administered",
    "numtotal_dose2_administered",
    "numtotal_dose3_administered",
    "numtotal_dose4_administered",
    "updated"
]

vacc_trimmed = vaccination.select(*cols_keep_vacc)

# 3.2: Inspect schema and sample
vacc_trimmed.printSchema()
display(vacc_trimmed.limit(5))
vacc_trimmed.count()


root
 |-- pruid: integer (nullable = true)
 |-- prename: string (nullable = true)
 |-- report_date: date (nullable = true)
 |-- as_of_date: date (nullable = true)
 |-- numtotal_all_administered: integer (nullable = true)
 |-- numtotal_dose1_administered: string (nullable = true)
 |-- numtotal_dose2_administered: string (nullable = true)
 |-- numtotal_dose3_administered: string (nullable = true)
 |-- numtotal_dose4_administered: integer (nullable = true)
 |-- updated: integer (nullable = true)



pruid,prename,report_date,as_of_date,numtotal_all_administered,numtotal_dose1_administered,numtotal_dose2_administered,numtotal_dose3_administered,numtotal_dose4_administered,updated
24,Quebec,2021-01-19,2021-01-18,164053,null,null,null,null,1
35,Ontario,2021-01-19,2021-01-18,262992,246799,16193,0,0,1
59,British Columbia,2021-01-19,2021-01-19,92369,null,null,null,null,1
48,Alberta,2021-01-19,2021-01-18,92315,null,null,null,null,1
47,Saskatchewan,2021-01-19,2021-01-18,24575,null,null,null,null,1


13230

### 3.3 Replace `"-"` or blank with `null`, then cast to correct types

- Integer fields:  
  `pruid`, `numtotal_all_administered`, `numtotal_dose1_administered`, `numtotal_dose2_administered`, `numtotal_dose3_administered`, `numtotal_dose4_administered`, `updated`  
- Date fields:  
  `report_date`, `as_of_date` (format `yyyy-MM-dd`)  
- Keep `prename` as string  


In [0]:
# 3.3.1: Replace "-" or blank strings with null
tmp2 = vacc_trimmed
for c in [
    "pruid",
    "numtotal_all_administered",
    "numtotal_dose1_administered",
    "numtotal_dose2_administered",
    "numtotal_dose3_administered",
    "numtotal_dose4_administered",
    "updated"
]:
    tmp2 = tmp2.withColumn(
        c,
        when((trim(col(c)) == "") | (col(c) == "-"), None).otherwise(col(c))
    )

# 3.3.2: Cast to correct types
vacc_cast = tmp2 \
    .withColumn("pruid", col("pruid").cast(IntegerType())) \
    .withColumn("numtotal_all_administered", col("numtotal_all_administered").cast(IntegerType())) \
    .withColumn("numtotal_dose1_administered", col("numtotal_dose1_administered").cast(IntegerType())) \
    .withColumn("numtotal_dose2_administered", col("numtotal_dose2_administered").cast(IntegerType())) \
    .withColumn("numtotal_dose3_administered", col("numtotal_dose3_administered").cast(IntegerType())) \
    .withColumn("numtotal_dose4_administered", col("numtotal_dose4_administered").cast(IntegerType())) \
    .withColumn("updated", col("updated").cast(IntegerType())) \
    .withColumn("report_date", to_date(col("report_date"), "yyyy-MM-dd")) \
    .withColumn("as_of_date", to_date(col("as_of_date"), "yyyy-MM-dd"))


In [0]:
vacc_cast.printSchema()
display(vacc_cast.limit(5))
vacc_cast.count()

root
 |-- pruid: integer (nullable = true)
 |-- prename: string (nullable = true)
 |-- report_date: date (nullable = true)
 |-- as_of_date: date (nullable = true)
 |-- numtotal_all_administered: integer (nullable = true)
 |-- numtotal_dose1_administered: integer (nullable = true)
 |-- numtotal_dose2_administered: integer (nullable = true)
 |-- numtotal_dose3_administered: integer (nullable = true)
 |-- numtotal_dose4_administered: integer (nullable = true)
 |-- updated: integer (nullable = true)



pruid,prename,report_date,as_of_date,numtotal_all_administered,numtotal_dose1_administered,numtotal_dose2_administered,numtotal_dose3_administered,numtotal_dose4_administered,updated
24,Quebec,2021-01-19,2021-01-18,164053,null,null,null,null,1
35,Ontario,2021-01-19,2021-01-18,262992,246799,16193,0,0,1
59,British Columbia,2021-01-19,2021-01-19,92369,null,null,null,null,1
48,Alberta,2021-01-19,2021-01-18,92315,null,null,null,null,1
47,Saskatchewan,2021-01-19,2021-01-18,24575,null,null,null,null,1


13230

### 3.4 Handle Missing Data

- Drop rows missing `pruid`, `report_date`, or `numtotal_all_administered`.  
- For dose columns and `updated`, fill null → 0.


In [0]:
# 3.4.1: Drop rows missing mandatory fields
vacc_nonull = vacc_cast.na.drop(subset=["pruid", "report_date", "numtotal_all_administered"])

# 3.4.2: Fill nulls for doses and updated
vacc_filled = vacc_nonull.fillna({
    "numtotal_dose1_administered": 0,
    "numtotal_dose2_administered": 0,
    "numtotal_dose3_administered": 0,
    "numtotal_dose4_administered": 0,
    "updated": 0
})


### 3.5 Filter Out Inconsistent Rows

If `(dose1 + dose2 + dose3 + dose4) > numtotal_all_administered`, drop them. Then compute `numtotal_other_or_unreported = numtotal_all_administered - sum(dose1..dose4)`.


In [0]:
from pyspark.sql.functions import expr

# 3.5.1: Compute sum of dose1–4
vacc_with_sum = vacc_filled.withColumn(
    "sum_dose1_to_4",
    col("numtotal_dose1_administered")
  + col("numtotal_dose2_administered")
  + col("numtotal_dose3_administered")
  + col("numtotal_dose4_administered")
)

# 3.5.2: Filter out bad rows
vacc_valid = vacc_with_sum.filter(
    col("sum_dose1_to_4") <= col("numtotal_all_administered")
)

# 3.5.3: Compute "other/unreported" column
vacc_clean = vacc_valid.withColumn(
    "numtotal_other_or_unreported",
    col("numtotal_all_administered") - col("sum_dose1_to_4")
)


### 3.6 Remove Duplicates Based on Natural Key

Natural key is `(pruid, report_date)`. Drop duplicates.


In [0]:
vacc_clean = vacc_clean.dropDuplicates(["pruid", "report_date"])


### 3.7 Final Check

Verify schema and row count in `vacc_clean`.


In [0]:
vacc_clean.printSchema()
print("Rows in vacc_clean:", vacc_clean.count())
display(vacc_clean.limit(5))


root
 |-- pruid: integer (nullable = true)
 |-- prename: string (nullable = true)
 |-- report_date: date (nullable = true)
 |-- as_of_date: date (nullable = true)
 |-- numtotal_all_administered: integer (nullable = true)
 |-- numtotal_dose1_administered: integer (nullable = false)
 |-- numtotal_dose2_administered: integer (nullable = false)
 |-- numtotal_dose3_administered: integer (nullable = false)
 |-- numtotal_dose4_administered: integer (nullable = false)
 |-- updated: integer (nullable = false)
 |-- sum_dose1_to_4: integer (nullable = false)
 |-- numtotal_other_or_unreported: integer (nullable = true)

Rows in vacc_clean: 13154


pruid,prename,report_date,as_of_date,numtotal_all_administered,numtotal_dose1_administered,numtotal_dose2_administered,numtotal_dose3_administered,numtotal_dose4_administered,updated,sum_dose1_to_4,numtotal_other_or_unreported
62,Nunavut,2021-01-25,2021-01-25,4458,0,0,0,0,1,0,4458
47,Saskatchewan,2021-01-26,2021-01-25,34080,0,0,0,0,1,0,34080
62,Nunavut,2021-03-09,2021-03-09,14943,0,0,0,0,1,0,14943
12,Nova Scotia,2021-03-15,2021-03-14,48077,0,0,0,0,1,0,48077
24,Quebec,2021-03-24,2021-03-23,1024713,0,0,0,0,1,0,1024713


### 4.1 Inspect Raw Data

Check schema and first rows. We only need:

- `REF_DATE` (format “YYYY-MM”)
- `GEO`      (province name)
- `VALUE`    (population number)


In [0]:
display(provpop.limit(5))



REF_DATE,GEO,DGUID,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
2018-01-01,Canada,2016A000011124,Persons,249,units,0,v1,1,36801579,null,null,null,0
2018-04-01,Canada,2016A000011124,Persons,249,units,0,v1,1,36903671,null,null,null,0
2018-07-01,Canada,2016A000011124,Persons,249,units,0,v1,1,37072620,null,null,null,0
2018-10-01,Canada,2016A000011124,Persons,249,units,0,v1,1,37259485,null,null,null,0
2019-01-01,Canada,2016A000011124,Persons,249,units,0,v1,1,37336956,null,null,null,0


## Step 4: Build **dim_province** (before joining population)

We need a table `dim_province(pruid, prname)` with no French name.  
We’ll union the `(pruid, prname)` pairs from `death_clean` and `vacc_clean`.


In [0]:
# 4.1: Provinces from death_clean
prov_from_death = death_clean.select(
    col("pruid"),
    col("prname")
).distinct()

# 4.2: Provinces from vacc_clean (rename prename → prname)
prov_from_vacc = vacc_clean.select(
    col("pruid"),
    col("prename").alias("prname")
).distinct()

# 4.3: Union and remove duplicates on pruid
dim_province = prov_from_death.union(prov_from_vacc).dropDuplicates(["pruid"])

# 4.4: Final check: schema and sample rows
dim_province.printSchema()
display(dim_province.orderBy("pruid").limit(10))
print("Rows in dim_province:", dim_province.count())


root
 |-- pruid: integer (nullable = true)
 |-- prname: string (nullable = true)



pruid,prname
1,Canada
10,Newfoundland and Labrador
11,Prince Edward Island
12,Nova Scotia
13,New Brunswick
24,Quebec
35,Ontario
46,Manitoba
47,Saskatchewan
48,Alberta


Rows in dim_province: 15


## Step 5: Clean & Build **dim_population**

Now that `dim_province(pruid, prname)` exists, we can join it to translated `provpop`.

1. Convert `REF_DATE → PopDate` (first of month).
2. Keep `(prname, PopDate, PopulationValue)`.
3. Join to `dim_province` on `prname` to bring in `pruid`.
4. Drop any rows missing keys.
5. Remove duplicates on `(pruid, PopDate)`.


In [0]:
from pyspark.sql.functions import concat, lit, to_date, col

# 4.1: First, look at distinct values of GEO in provpop and prname in dim_province
print("Distinct GEO in provpop:")
display(provpop.select("GEO").distinct().orderBy("GEO"))

print("Distinct prname in dim_province:")
display(dim_province.select("prname").distinct().orderBy("prname"))


Distinct GEO in provpop:


GEO
Alberta
British Columbia
Canada
Manitoba
New Brunswick
Newfoundland and Labrador
Northwest Territories
Nova Scotia
Nunavut
Ontario


Distinct prname in dim_province:


prname
Alberta
British Columbia
Canada
Manitoba
New Brunswick
Newfoundland and Labrador
Northwest Territories
Nova Scotia
Nunavut
Ontario


In [0]:
from pyspark.sql.functions import concat, lit, to_date, col

# 4.2.1: Join provpop to dim_province (inner join ensures only matching provinces)
provpop_with_pruid = provpop.join(
    dim_province,
    provpop.GEO == dim_province.prname,
    how="inner"
)

# Verify schema and row count
provpop_with_pruid.printSchema()
print("Rows in dim_population:", provpop_with_pruid.count())

# Show a few rows
display(provpop_with_pruid.limit(10))

root
 |-- REF_DATE: date (nullable = true)
 |-- GEO: string (nullable = true)
 |-- DGUID: string (nullable = true)
 |-- UOM: string (nullable = true)
 |-- UOM_ID: integer (nullable = true)
 |-- SCALAR_FACTOR: string (nullable = true)
 |-- SCALAR_ID: integer (nullable = true)
 |-- VECTOR: string (nullable = true)
 |-- COORDINATE: integer (nullable = true)
 |-- VALUE: integer (nullable = true)
 |-- STATUS: string (nullable = true)
 |-- SYMBOL: string (nullable = true)
 |-- TERMINATED: string (nullable = true)
 |-- DECIMALS: integer (nullable = true)
 |-- pruid: integer (nullable = true)
 |-- prname: string (nullable = true)

Rows in dim_population: 406


REF_DATE,GEO,DGUID,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS,pruid,prname
2025-01-01,Nova Scotia,2016A000212,Persons,249,units,0,v9,4,1079627,null,null,null,0,12,Nova Scotia
2025-01-01,Saskatchewan,2016A000247,Persons,249,units,0,v14,9,1250909,null,null,null,0,47,Saskatchewan
2025-01-01,Canada,2016A000011124,Persons,249,units,0,v1,1,41528680,null,null,null,0,1,Canada
2025-01-01,New Brunswick,2016A000213,Persons,249,units,0,v10,5,858963,null,null,null,0,13,New Brunswick
2025-01-01,Alberta,2016A000248,Persons,249,units,0,v15,10,4960097,null,null,null,0,48,Alberta
2025-01-01,Northwest Territories,2016A000261,Persons,249,units,0,v6,14,45074,null,null,null,0,61,Northwest Territories
2025-01-01,Ontario,2016A000235,Persons,249,units,0,v12,7,16182641,null,null,null,0,35,Ontario
2025-01-01,British Columbia,2016A000259,Persons,249,units,0,v3,11,5722318,null,null,null,0,59,British Columbia
2025-01-01,Newfoundland and Labrador,2016A000210,Persons,249,units,0,v2,2,545579,null,null,null,0,10,Newfoundland and Labrador
2025-01-01,Quebec,2016A000224,Persons,249,units,0,v11,6,9111629,null,null,null,0,24,Quebec


In [0]:

# 4.2.2: Convert REF_DATE to PopDate and cast VALUE to integer
dim_population = provpop_with_pruid \
    .withColumnRenamed("REF_DATE", "PopDate") \
    .withColumn("PopulationValue", col("VALUE").cast("integer")) \
    .select(
        col("pruid"),      # from dim_province
        col("prname"),     # English name from dim_province
        col("PopDate"),
        col("PopulationValue")
    )

# Verify schema and row count
dim_population.printSchema()
print("Rows in dim_population:", dim_population.count())

# Show a few rows
display(dim_population.orderBy("pruid", "PopDate").limit(10))

root
 |-- pruid: integer (nullable = true)
 |-- prname: string (nullable = true)
 |-- PopDate: date (nullable = true)
 |-- PopulationValue: integer (nullable = true)

Rows in dim_population: 406


pruid,prname,PopDate,PopulationValue
1,Canada,2018-01-01,36801579
1,Canada,2018-04-01,36903671
1,Canada,2018-07-01,37072620
1,Canada,2018-10-01,37259485
1,Canada,2019-01-01,37336956
1,Canada,2019-04-01,37437243
1,Canada,2019-07-01,37618495
1,Canada,2019-10-01,37828162
1,Canada,2020-01-01,37928208
1,Canada,2020-04-01,38006941


In [0]:
# 4.2.3: Drop any rows missing keys or values
dim_population = dim_population.na.drop(subset=["pruid", "PopDate", "PopulationValue"])

# 4.2.4: Remove duplicates on (pruid, PopDate)
dim_population = dim_population.dropDuplicates(["pruid", "PopDate"])

In [0]:
dim_population = dim_population.drop("prname")

In [0]:
# Verify schema and row count
dim_population.printSchema()
print("Rows in dim_population:", dim_population.count())

# Show a few rows
display(dim_population.orderBy("pruid", "PopDate").limit(10))


root
 |-- pruid: integer (nullable = true)
 |-- PopDate: date (nullable = true)
 |-- PopulationValue: integer (nullable = true)

Rows in dim_population: 406


pruid,PopDate,PopulationValue
1,2018-01-01,36801579
1,2018-04-01,36903671
1,2018-07-01,37072620
1,2018-10-01,37259485
1,2019-01-01,37336956
1,2019-04-01,37437243
1,2019-07-01,37618495
1,2019-10-01,37828162
1,2020-01-01,37928208
1,2020-04-01,38006941


## Step 6: Build **dim_date**

We need a calendar table with:
- `FullDate` (all unique dates)
- `Year`, `Month`, `Day`
- `WeekOfYear`
- `IsWeekEnd` (1 if Saturday, else 0)
- `MonthStart` (first day of that month)

We’ll pull dates from:
- `death_clean.date`
- `vacc_clean.report_date`
- `dim_population.PopDate`


In [0]:
from pyspark.sql.functions import year, month, dayofmonth, weekofyear, dayofweek, trunc

# 6.1: Collect all distinct dates
dates_from_death = death_clean.select(col("date").alias("FullDate"))
dates_from_vacc  = vacc_clean.select(col("report_date").alias("FullDate"))
dates_from_pop   = dim_population.select(col("PopDate").alias("FullDate"))

all_dates = dates_from_death.union(dates_from_vacc).union(dates_from_pop).dropDuplicates(["FullDate"])

# 6.2: Compute date attributes
dim_date = all_dates \
    .withColumn("Year", year(col("FullDate"))) \
    .withColumn("Month", month(col("FullDate"))) \
    .withColumn("Day", dayofmonth(col("FullDate"))) \
    .withColumn("WeekOfYear", weekofyear(col("FullDate"))) \
    .withColumn(
        "IsWeekEnd",
        when(dayofweek(col("FullDate")) == 7, 1).otherwise(0)  # Saturday → 7
    ) \
    .withColumn(
        "MonthStart",
        trunc(col("FullDate"), "month")
    )




In [0]:
# 6.3: Final check
dim_date.printSchema()
print("Rows in dim_date:", dim_date.count())
display(dim_date.orderBy("FullDate").filter("IsWeekEnd = 0").limit(10))

root
 |-- FullDate: date (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Day: integer (nullable = true)
 |-- WeekOfYear: integer (nullable = true)
 |-- IsWeekEnd: integer (nullable = false)
 |-- MonthStart: date (nullable = true)

Rows in dim_date: 1078


FullDate,Year,Month,Day,WeekOfYear,IsWeekEnd,MonthStart
2018-01-01,2018,1,1,1,0,2018-01-01
2018-04-01,2018,4,1,13,0,2018-04-01
2018-07-01,2018,7,1,26,0,2018-07-01
2018-10-01,2018,10,1,40,0,2018-10-01
2019-01-01,2019,1,1,1,0,2019-01-01
2019-04-01,2019,4,1,14,0,2019-04-01
2019-07-01,2019,7,1,27,0,2019-07-01
2019-10-01,2019,10,1,40,0,2019-10-01
2020-01-01,2020,1,1,1,0,2020-01-01
2020-04-01,2020,4,1,14,0,2020-04-01


## Step 7: Build **fact_covid**

From `death_clean`, select:
- `pruid`
- `date`
- `reporting_year`
- `reporting_week`
- `totalcases`
- `numtotal_last7` → alias as `numcases_weekly`
- `numdeaths_last7` → alias as `numdeaths_weekly`
- `ratecases_last7`
- `ratedeaths_last7`

We’ll ensure no duplicates on (pruid, date).


In [0]:
from pyspark.sql.functions import col

fact_covid = death_clean.select(
    col("pruid"),
    col("date"),
    col("reporting_year"),
    col("reporting_week"),
    col("totalcases"),
    col("numtotal_last7").alias("numcases_weekly"),
    col("numdeaths_last7").alias("numdeaths_weekly"),
    col("ratecases_last7"),
    col("ratedeaths_last7")
)

# 7.1: Check uniqueness on (pruid, date)
dup_count_covid = fact_covid.groupBy("pruid", "date").count().filter(col("count") > 1).count()
print(f"Duplicates in fact_covid on (pruid, date): {dup_count_covid}")

# 7.2: Sample rows
display(fact_covid.orderBy("pruid", "date").limit(5))


Duplicates in fact_covid on (pruid, date): 0


pruid,date,reporting_year,reporting_week,totalcases,numcases_weekly,numdeaths_weekly,ratecases_last7,ratedeaths_last7
1,2020-02-08,2020,6,8,4,0,0.01,0.0
1,2020-02-15,2020,7,8,0,0,0.0,0.0
1,2020-02-22,2020,8,11,3,0,0.01,0.0
1,2020-02-29,2020,9,25,14,0,0.03,0.0
1,2020-03-07,2020,10,63,38,0,0.09,0.0


## Step 8: Build **fact_vaccination**

From `vacc_clean`, select:
- `pruid`
- `report_date` → alias as `date`
- `numtotal_all_administered`
- `numtotal_dose1_administered`
- `numtotal_dose2_administered`
- `numtotal_dose3_administered`
- `numtotal_dose4_administered`
- `numtotal_other_or_unreported`
- `updated`

Ensure no duplicates on (pruid, date).


In [0]:
fact_vaccination = vacc_clean.select(
    col("pruid"),
    col("report_date").alias("date"),
    col("numtotal_all_administered"),
    col("numtotal_dose1_administered"),
    col("numtotal_dose2_administered"),
    col("numtotal_dose3_administered"),
    col("numtotal_dose4_administered"),
    col("numtotal_other_or_unreported"),
    col("updated")
)

# 8.1: Check uniqueness on (pruid, date)
dup_count_vacc = fact_vaccination.groupBy("pruid", "date").count().filter(col("count") > 1).count()
print(f"Duplicates in fact_vaccination on (pruid, date): {dup_count_vacc}")

# 8.2: Sample rows
display(fact_vaccination.orderBy("pruid", "date").limit(5))


Duplicates in fact_vaccination on (pruid, date): 0


pruid,date,numtotal_all_administered,numtotal_dose1_administered,numtotal_dose2_administered,numtotal_dose3_administered,numtotal_dose4_administered,numtotal_other_or_unreported,updated
1,2021-01-19,685572,257333,17580,0,0,410659,1
1,2021-01-20,727127,263643,21506,0,0,441978,1
1,2021-01-21,765623,268932,28605,0,0,468086,1
1,2021-01-22,798940,271367,32913,0,0,494660,1
1,2021-01-23,817801,273774,35896,0,0,508131,1


In [0]:
# 10.1: Write dim_province to ADLS
dim_province.write.mode("overwrite") \
  .option("header", "true") \
  .csv(
    f"abfss://covid-project-data@{storage_account}.dfs.core.windows.net/transformed-data/dim_province"
  )

# 10.2: Write dim_date to ADLS
dim_date.write.mode("overwrite") \
  .option("header", "true") \
  .csv(
    f"abfss://covid-project-data@{storage_account}.dfs.core.windows.net/transformed-data/dim_date"
  )

# 10.3: Write dim_population to ADLS
dim_population.write.mode("overwrite") \
  .option("header", "true") \
  .csv(
    f"abfss://covid-project-data@{storage_account}.dfs.core.windows.net/transformed-data/dim_population"
  )

# 10.4: Write fact_covid to ADLS
fact_covid.write.mode("overwrite") \
  .option("header", "true") \
  .csv(
    f"abfss://covid-project-data@{storage_account}.dfs.core.windows.net/transformed-data/fact_covid"
  )

# 10.5: Write fact_vaccination to ADLS
fact_vaccination.write.mode("overwrite") \
  .option("header", "true") \
  .csv(
    f"abfss://covid-project-data@{storage_account}.dfs.core.windows.net/transformed-data/fact_vaccination"
  )
